In [2]:
import graphviz
import os
from uuid import uuid4
import numpy as np
from math import floor, sqrt
from sklearn.ensemble import RandomForestClassifier
import time

# Problem 1
class Question:
    """Questions to use in construction and display of Decision Trees.
    Attributes:
        column (int): which column of the data this question asks
        value (int/float): value the question asks about
        features (str): name of the feature asked about
    Methods:
        match: returns boolean of if a given sample answered T/F"""
    
    def __init__(self, column, value, feature_names):
        self.column = column
        self.value = value
        self.features = feature_names[self.column]
    
    def match(self,sample):
        """Returns T/F depending on how the sample answers the question
        Parameters:
            sample ((n,), ndarray): New sample to classify
        Returns:
            (bool): How the sample compares to the question"""
        #compare the sample to the value at the correct column

        return sample[self.column] >= self.value
        
    def __repr__(self):
        return "Is %s >= %s?" % (self.features, str(self.value))
    

In [3]:
def partition(data,question):
    """Splits the data into left (true) and right (false)
    Parameters:
        data ((m,n), ndarray): data to partition
        question (Question): question to split on
    Returns:
        left ((j,n), ndarray): Portion of the data matching the question
        right ((m-j, n), ndarray): Portion of the data NOT matching the question
    """
    #use our question method to answer the question and portion accordingly
    left = data[data[:, question.column] >= question.value, :]
    right = data[data[:, question.column] < question.value, :]

    return left, right
    

In [4]:
#Problem 2    
def gini(data):
    """Return the Gini impurity of given array of data.
    Parameters:
        data (ndarray): data to examine
    Returns:
        (float): Gini impurity of the data"""
    #get the number of samples
    nsamples = len(data)
    
    #get the occurences of each label
    label_occurrences = {}
    for label in data[:, -1]:
        if label in label_occurrences:
            label_occurrences[label] += 1
        else:
            label_occurrences[label] = 1
    #calculate the gini index
    gini = 1
    for occurrence in label_occurrences.values():
        gini -= (occurrence/nsamples)**2
    
    return gini

In [5]:
def info_gain(left,right,G):
    """Return the info gain of a partition of data.
    Parameters:
        left (ndarray): left split of data
        right (ndarray): right split of data
        G (float): Gini impurity of unsplit data
    Returns:
        (float): info gain of the data"""
    #get the number of data points in each partition and in total
    lp, rp = len(left), len(right)
    alltogether = lp+rp
    #calculate the summation term for each i
    first_term = (lp/alltogether)*gini(left)
    second_term = (rp/alltogether)*gini(right)

    #return the summation value
    return (G - (first_term + second_term))

In [7]:
def test_gains():
    "Test the information gain of a partition of data"
    animals = np.loadtxt('animals.csv', delimiter=',')
    print(gini(animals))

    print(info_gain(animals[:50], animals[50:], gini(animals)))
test_gains()

0.4758
0.14580000000000004


In [8]:
# Problem 3, Problem 7
def find_best_split(data, feature_names, min_samples_leaf=5, random_subset=False):
    """Find the optimal split
    Parameters:
        data (ndarray): Data in question
        feature_names (list of strings): Labels for each column of data
        min_samples_leaf (int): minimum number of samples per leaf
        random_subset (bool): for Problem 7
    Returns:
        (float): Best info gain
        (Question): Best question"""
    #get the best gains and question
    best_gains = 0
    best_question = None

    
    options = feature_names[:-1]
    nfeatures = len(options)
    G = gini(data)
    #account for random_subset
    if random_subset:
        n = len(options)
        randos = floor(sqrt(n))
        rando_ind = np.random.randint(low=0, high=len(options), size=randos)

    for col in range(nfeatures):
        if random_subset and col not in rando_ind:
            continue
        split_ops = np.unique(data[:, col])
        #get the unique classification options
        for op in split_ops:
            split_q = Question(column=col, value=op, feature_names=options)
            left, right = partition(data, split_q)

            if len(left) < min_samples_leaf or len(right) < min_samples_leaf:
                continue
            
            #calculate the information gain of each option
            gains = info_gain(left, right, G)
            if gains > best_gains:
                best_gains = gains
                best_question = split_q

    return best_gains, best_question

In [12]:
def test_best_split():
    "Find the optimal split for the data that maximizes information gain and give the gain"
    # Load in the data
    animals = np.loadtxt('animals.csv', delimiter=',')
    # Load in feature names
    features = np.loadtxt('animal_features.csv', delimiter=',', dtype=str, comments=None)
    # Load in sample names
    names = np.loadtxt('animal_names.csv', delimiter=',', dtype=str)
    print(find_best_split(animals, features))
test_best_split()

(0.12259833679833687, Is # legs/tentacles >= 2.0?)


In [13]:
# Problem 4
class Leaf:
    """Tree leaf node
    Attribute:
        prediction (dict): Dictionary of labels at the leaf"""
    def __init__(self,data):

        #account for there only being one entry
        if len(data.shape) == 1:
            predict = {data[-1] : 1}
            self.prediction = predict
        else:
            label_occurr = {}
            #count the number of times each label appears
            for label in data[:,-1]:
                if label in label_occurr:
                    label_occurr[label] += 1
                else:
                    label_occurr[label] = 1  
            
            self.prediction = label_occurr

In [14]:
class Decision_Node:
    """Tree node with a question
    Attributes:
        question (Question): Question associated with node
        left (Decision_Node or Leaf): child branch
        right (Decision_Node or Leaf): child branch"""
    def __init__(self, question, left_branch, right_branch):
        #initialize the attributes
        self.question = question
        self.left_branch = left_branch
        self.right_branch = right_branch
        

In [15]:
## Code to draw a tree
def draw_node(graph, my_tree):
    """Helper function for drawTree"""
    node_id = uuid4().hex
    #If it's a leaf, draw an oval and label with the prediction
    if isinstance(my_tree, Leaf):
        graph.node(node_id, shape="oval", label="%s" % my_tree.prediction)
        return node_id
    else: #If it's not a leaf, make a question box
        graph.node(node_id, shape="box", label="%s" % my_tree.question)
        left_id = draw_node(graph, my_tree.left)
        graph.edge(node_id, left_id, label="T")
        right_id = draw_node(graph, my_tree.right)    
        graph.edge(node_id, right_id, label="F")
        return node_id

def draw_tree(my_tree):
    """Draws a tree"""
    #Remove the files if they already exist
    for file in ['Digraph.gv','Digraph.gv.pdf']:
        if os.path.exists(file):
            os.remove(file)
    graph = graphviz.Digraph(comment="Decision Tree")
    draw_node(graph, my_tree)
    graph.render(view=True) #This saves Digraph.gv and Digraph.gv.pdf

In [16]:
# Prolem 5
def build_tree(data, feature_names, min_samples_leaf=5, max_depth=4, current_depth=0, random_subset=False):
    """Build a classification tree using the classes Decision_Node and Leaf
    Parameters:
        data (ndarray)
        feature_names(list or array)
        min_samples_leaf (int): minimum allowed number of samples per leaf
        max_depth (int): maximum allowed depth
        current_depth (int): depth counter
        random_subset (bool): whether or not to train on a random subset of features
    Returns:
        Decision_Node (or Leaf)"""
    #if another split will takes us below our min samples, end the recursion
    if len(data) < 2 * min_samples_leaf:
        return Leaf(data)
    
    #iterate recursively
    best_gains, best_question = find_best_split(data, feature_names, min_samples_leaf=min_samples_leaf, random_subset=random_subset)

    #end if we don't improve or we are at max depth
    if best_gains == 0 or current_depth >= max_depth:
        return Leaf(data)

    left, right = partition(data, best_question)
    
    #split and build nodes for each side if we do split
    leftie = build_tree(left, feature_names,min_samples_leaf=min_samples_leaf, max_depth=max_depth, current_depth=current_depth+1, random_subset=random_subset)
    rightie = build_tree(right, feature_names, min_samples_leaf=min_samples_leaf, max_depth=max_depth, current_depth=current_depth+1, random_subset=random_subset)

    return Decision_Node(best_question, leftie, rightie)

In [17]:
# Problem 6
def predict_tree(sample, my_tree):
    """Predict the label for a sample given a pre-made decision tree
    Parameters:
        sample (ndarray): a single sample
        my_tree (Decision_Node or Leaf): a decision tree
    Returns:
        Label to be assigned to new sample"""

    #end the recursion if we are at a leaf
    if isinstance(my_tree, Leaf):
        return max(my_tree.prediction, key=my_tree.prediction.get)

    #continue by recursion at nodes and parse through tree based upon each node's decision
    if my_tree.question.match(sample):
        return predict_tree(sample, my_tree.left_branch)
        
    else:
        return predict_tree(sample, my_tree.right_branch)

In [18]:
def analyze_tree(dataset,my_tree):
    """Test how accurately a tree classifies a dataset
    Parameters:
        dataset (ndarray): Labeled data with the labels in the last column
        tree (Decision_Node or Leaf): a decision tree
    Returns:
        (float): Proportion of dataset classified correctly"""

    nsamples = len(dataset)
    ngoodies = 0

    for sample in dataset:
        #see if the prediction is correct
        if predict_tree(sample, my_tree) == sample[-1]:
            ngoodies += 1
        else:
            pass
    #calculate percentage of correct labels
    return ngoodies / nsamples

In [20]:
def test_tree():
    # Load in the data
    animals = np.loadtxt('animals.csv', delimiter=',')
    # Load in feature names
    animal_features = np.loadtxt('animal_features.csv', delimiter=',', dtype=str, comments=None)
    # Load in sample names
    names = np.loadtxt('animal_names.csv', delimiter=',', dtype=str)
    np.random.shuffle(animals)
    training_set, test_set = animals[:80, :], animals[80:, :]
    my_tree = build_tree(data=training_set, feature_names=animal_features)
    print(analyze_tree(dataset=test_set, my_tree=my_tree))
test_tree()

0.85


In [21]:
# Problem 7
def predict_forest(sample, forest):
    """Predict the label for a new sample, given a random forest
    Parameters:
        sample (ndarray): a single sample
        forest (list): a list of decision trees
    Returns:
        Label to be assigned to new sample"""
    
    #get the prediction of each tree in the forest
    tree_labels = [predict_tree(sample, tree) for tree in forest]

    #get the most common label
    return max(set(tree_labels), key=tree_labels.count)

In [23]:
def analyze_forest(dataset,forest):
    """Test how accurately a forest classifies a dataset
    Parameters:
        dataset (ndarray): Labeled data with the labels in the last column
        forest (list): list of decision trees
    Returns:
        (float): Proportion of dataset classified correctly"""
    nsamples = len(dataset)
    goodies = 0
    for sample in dataset:
        if predict_forest(sample, forest) == sample[-1]:
            #see if our forest predicted correctly or not
            good = 1
        else:
            good = 0
        goodies += good

    #get the percentage of samples labeled correctly
    return goodies / nsamples

In [35]:
# Problem 8
def prob8():
    """Use the file parkinsons.csv to analyze a 5 tree forest.
    
    Create a forest with 5 trees and train on 100 random samples from the dataset.
    Use 30 random samples to test using analyze_forest() and SkLearn's 
    RandomForestClassifier.
    
    Create a 5 tree forest using 80% of the dataset and analzye using 
    RandomForestClassifier.
    
    Return three tuples, one for each test.
    
    Each tuple should include the accuracy and time to run: (accuracy, running time) 
    """
    #load the data
    park_dat = np.loadtxt('parkinsons.csv', delimiter=',', dtype=float, comments=None)
    park_feats = np.loadtxt('parkinsons_features.csv', delimiter=',', dtype=str, comments=None)

    #don't use participant ID
    park_dat = park_dat[:, 1:]
    park_feats = park_feats[1:]

    #shuffle the data
    np.random.shuffle(park_dat)

    #use a subset
    train = park_dat[:100, :]
    test = park_dat[100:130, :]

    #use our forest class that we created on subset
    class_start= time.time()
    classy_forest = [build_tree(data=train, feature_names=park_feats, min_samples_leaf=15,max_depth=4, random_subset=True) for i in range(5)]
    class_acc = analyze_forest(dataset=test, forest=classy_forest)
    class_time = time.time() - class_start

    #use sklearn on subset
    sklearn_start = time.time()
    learned_forest = RandomForestClassifier(n_estimators=5, max_depth=4, min_samples_leaf=15)
    learned_forest.fit(train[:, :-1], train[:, -1])
    learned_acc = learned_forest.score(test[:, :-1], test[:, -1])
    learned_time = time.time() - sklearn_start

    #use sklearn on whole thing
    nsamples = len(park_dat)
    trainees = floor(nsamples*.8)
    train, test = park_dat[:trainees, :], park_dat[trainees:, :]
    bigsklearn_start = time.time()
    bigsklearn = RandomForestClassifier()
    bigsklearn.fit(train[:, :-1], train[:, -1])
    bigsklearn_acc = bigsklearn.score(test[:, :-1], test[:, -1])
    bigsklearn_time = time.time() - bigsklearn_start

    return (class_acc, class_time), (learned_acc, learned_time), (bigsklearn_acc, bigsklearn_time)
prob8()

((0.8, 0.11325216293334961),
 (0.7333333333333333, 0.0045659542083740234),
 (0.7355371900826446, 0.22952008247375488))